In [25]:
!pip install selenium-wire

In [26]:
#Importamos las librerías
import requests
import json
import time
import pandas as pd
from seleniumwire import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.remote.command import Command

Para encontrar el establecimiento ingresamos su razón social y el código postal.
Para pruebas pueden usar la razón social "GALLAGHER-KAISER DE MEXICO" y el código postal "66269".

In [2]:
nombre_original = input('INGRESE EL NOMBRE DE LA PYME: ')
nombre = (nombre_original.strip()).replace(' ','+')
codigo_postal = input('INGRESE EL CÓDIGO POSTAL DE LA PYME: ')

INGRESE EL NOMBRE DE LA PYME: DISTRIBUIDORA LIEVANO
INGRESE EL CÓDIGO POSTAL DE LA PYME: 29930


Esto nos dará algunas opciones, elegiremos la correcta con el número entero que le corresponda (en este caso 1 ó 2).

In [6]:
url_opciones = ('https://www.inegi.org.mx/app/mapa/componente/Espacial.ashx?metodo=ArmabusquedaEstablecimientos&Ac_eco%5B0%5D%5Bsec%5D=&Ac_eco%5B1%5D%5Bsubsec%5D=&Ac_eco%5B2%5D%5Brama%5D=&Ac_eco%5B3%5D%5Bsubra%5D=&Ac_eco%5B4%5D%5Bclase%5D=&Tama%C3%B1o=&Cve_geo%5B0%5D%5Bent%5D=&Cve_geo%5B1%5D%5Bmun%5D=&Cve_geo%5B2%5D%5Bloc%5D=&Txt_gral='+
'&Txt_nom='+nombre+
'&Txt_calle=&Txt_colonia='+
'&Txt_cp='+codigo_postal+
'&Txt_vc=&Txt_prod=&Txt_paisexp=&Txt_paisimp=&Txt_camara=&Txt_mts=&Txt_radio=&Txt_coord=&idee=&Pagini=1&Pagfin=50')

respuesta = requests.post(url_opciones)
opciones = respuesta.json()['obj'][:-1]

num_opcion = 1
for opcion in opciones:
    nombre_establecimiento = opcion[5]
    print(str(num_opcion)+'. '+nombre_establecimiento)
    num_opcion += 1

establecimiento_correcto = int(input('ELIJA EL ESTABLECIMIENTO CORRECTO: '))


1. OFICINA DE DISTRIBUIDORA LIEVANO
ELIJA EL ESTABLECIMIENTO CORRECTO: 1


Ya con esto puedo obtener los datos del establecimiento.

In [7]:
codigo_1_establecimiento = opciones[establecimiento_correcto - 1][0]
codigo_2_establecimiento = opciones[establecimiento_correcto - 1][3]
url_datos_establecimiento_correcto = ('https://www.inegi.org.mx/app/mapa/componente/APIGeografico/infowindowDenue/'+
                                      codigo_1_establecimiento +'/DENUE/'+codigo_2_establecimiento)
respuesta_final = requests.get(url_datos_establecimiento_correcto)
id_establecimiento = respuesta_final.json()[0]["d1"]
cve_ent = respuesta_final.json()[0]["d2"]
entidad = respuesta_final.json()[0]["d3"]
cve_mun = respuesta_final.json()[0]["d4"]
municipio = respuesta_final.json()[0]["d5"]
cve_loc = respuesta_final.json()[0]["d6"]
localidad = respuesta_final.json()[0]["d7"]
nom_estab = respuesta_final.json()[0]["d8"]
raz_social = respuesta_final.json()[0]["d9"]
codigo_act = respuesta_final.json()[0]["d11"]
nombre_act = respuesta_final.json()[0]["d12"]
per_ocu = respuesta_final.json()[0]["d14"]
tipo_vial = respuesta_final.json()[0]["d15"]
nom_vial = respuesta_final.json()[0]["d16"]
numero_ext = respuesta_final.json()[0]["d17"]
edificio = respuesta_final.json()[0]["d18"]
numero_int = respuesta_final.json()[0]["d19"]
tipo_asent = respuesta_final.json()[0]["d20"]
nomb_asent = respuesta_final.json()[0]["d21"]
nom_CenCom = respuesta_final.json()[0]["d22"]
num_local = respuesta_final.json()[0]["d23"]
cod_postal = respuesta_final.json()[0]["d24"]
ageb = respuesta_final.json()[0]["d25"]
manzana = respuesta_final.json()[0]["d26"]
telefono = respuesta_final.json()[0]["d27"]
correoelec = respuesta_final.json()[0]["d28"]
www = respuesta_final.json()[0]["d30"]
fecha_alta = respuesta_final.json()[0]["d33"]
latitud = respuesta_final.json()[0]["d34"]
longitud = respuesta_final.json()[0]["d35"]
tipo_asent = respuesta_final.json()[0]["d57"]
tipoCenCom = respuesta_final.json()[0]["d58"]
tipo_v_e_1 = respuesta_final.json()[0]["d61"]
nom_v_e_1 = respuesta_final.json()[0]["d62"]
tipo_v_e_2 = respuesta_final.json()[0]["d63"]
nom_v_e_2 = respuesta_final.json()[0]["d64"]
tipo_v_e_3 = respuesta_final.json()[0]["d65"]
nom_v_e_3 = respuesta_final.json()[0]["d66"]
clee = respuesta_final.json()[0]["d68"]

Entre los datos tenemos (según la documentación técnica de la DENUE).

In [8]:
df = pd.read_excel('diccionario_variables_DENUE.xlsx')
df['valor_de_ejemplo'] = [id_establecimiento, cve_ent, entidad, cve_mun, municipio, cve_loc, localidad, nom_estab, raz_social,codigo_act,
                          nombre_act, per_ocu, tipo_vial, nom_vial, numero_ext, edificio, numero_int, tipo_asent, nomb_asent,
                          nom_CenCom, num_local, cod_postal, ageb, manzana, telefono, correoelec, www, fecha_alta, latitud,
                          longitud, tipo_asent, tipoCenCom, tipo_v_e_1, nom_v_e_1, tipo_v_e_2, nom_v_e_2, tipo_v_e_3, 
                          nom_v_e_3, clee]
df

,nombre_json,nombre_variable,descripcion,valor_de_ejemplo
0,d1,id,"Número de identificación del DENUE, es una cl...",483488
1,d2,cve_ent,Clave que identifica a la entidad federativa e...,07
2,d3,entidad,Es el sustantivo propio que identifica a la en...,CHIAPAS
3,d4,cve_mun,"Clave que identifica al Municipio estadístico,...",109
4,d5,municipio,Es el sustantivo propio que identifica al Muni...,Yajalón
5,d6,cve_loc,Clave que identifica a la localidad estadístic...,0001
6,d7,localidad,Es el sustantivo propio que identifica a la lo...,Yajalón
7,d8,nom_estab,Es el nombre comercial o nombre exterior con e...,OFICINA DE DISTRIBUIDORA LIEVANO
8,d9,raz_social,Es la forma con que está legalmente constituid...,
9,d11,codigo_act,La clasificación de las actividades desarrolla...,541211


In [27]:
#Abrimos el chromedriver para poder entrar al google maps y extraer la info

options = Options()

#Si se desea ver el navegador de google chrome que se está ejecutando dejar comentada toda esta sección
#-------------------------------------------------------------------------------------------------------#
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-infobars')
options.add_argument('--disable-browser-side-navigation')
#--------------------------------------------------------------------------------------------------------#

driver = webdriver.Chrome("chromedriver.exe", options=options)


In [52]:
#Primero ubico el mapa en una latitud y longitud cercana a donde esta en el maps
driver.get('https://www.google.com.ec/maps/@'+latitud+','+longitud+',19z?hl=es')
#Luego la busco por su nombre
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(nombre_original)
#Doy Enter
driver.find_element(By.XPATH, '//*[@id="searchboxinput"]').send_keys(Keys.ENTER)
#Espero a que cargue la página
time.sleep(10)
#Entro en las reseñas
driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/div[2]/span[2]/span[1]/button').click()
#Espero a que cargue la página
time.sleep(10)

#Extraigo el link de la api donde estan todos los datos de reseñas
url_resenias = "ERROR"
for request in driver.requests:
    if request.response:
        if "https://www.google.com.ec/maps/preview/review/listentitiesreviews?authuser" in str(request.url):
            url_resenias = str(request.url)

#Ya con el link puedo acceder a la data
resenias = requests.get(url_resenias)
#Así se ve la data, tengo que ordenarla
print(str(resenias.content))

#Cierro el navegador
driver.close()

b')]}\'\n[null,null,[[["https://www.google.com/maps/contrib/100641398209875934145?hl\\u003des-419","Miguel Angel","https://lh3.googleusercontent.com/a/ALm5wu01KEa43YcAN5gRav4O2AgqgrFDj6hhYDQi_5e0\\u003ds120-c-c0x00000000-cc-rp-mo-ba3-br100","0ahUKEwiboYvbzPL6AhVkTmwGHWG3B58Q4h4IAygA"],"Hace un a\xc3\xb1o",null,"Excelente",5,null,"100641398209875934145",["https://www.google.com/local/review/rap/report?postId\\u003dChdDSUhNMG9nS0VJQ0FnSUR5LWNTcWlnRRAB\\u0026entityid\\u003dChdDSUhNMG9nS0VJQ0FnSUR5LWNTcWlnRRItChZDSUhNMG9nS0VJQ0FnSUR5LWNTcVNnEhNDZ3dJbGIyeGdnWVF5SWZkb0FFGi4KF0NJSE0wb2dLRUlDQWdJRHktY1NxeWdFEhNDZ3dJbGIyeGdnWVF5SWZkb0FFIhIJ6-I714OF8oURxad8apqgwwYqE0Nnd0lsYjJ4Z2dZUXlJZmRvQUU\\u0026wv\\u003d1\\u0026d\\u003d286732320","Marcar como inadecuado",null,"0ahUKEwiboYvbzPL6AhVkTmwGHWG3B58QoykIBigD"],null,null,"ChdDSUhNMG9nS0VJQ0FnSUR5LWNTcWlnRRAB","0ahUKEwiboYvbzPL6AhVkTmwGHWG3B58Q4R4IAigA",[null,[[5,null,1],10,12,"https://lh3.googleusercontent.com/a/ALm5wu01KEa43YcAN5gRav4O2AgqgrFDj6hhYD